## Temiloluwa Adeoti, 303008

In [1]:
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np
import math


from sklearn import tree
iris_data = load_iris(return_X_y=False)
breast_cancer_data = load_breast_cancer(return_X_y=False)

X_iris,y_iris = iris_data['data'],iris_data['target']
X_breast,y_breast = breast_cancer_data['data'],breast_cancer_data['target']

X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(
            X_iris, y_iris, test_size=0.3, random_state=2019)

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(
            X_breast, y_breast, test_size=0.3, random_state=2019)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train_i,  y_train_i)
print(f"Accuracy score by Sklearn for Iris Dataset {clf.score(X_test_i,y_test_i)}")

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train_b,  y_train_b)
print(f"Accuracy score by Sklearn for Breast Cancer Dataset {clf.score(X_test_b,y_test_b)}")

Accuracy score by Sklearn for Iris Dataset 0.9555555555555556
Accuracy score by Sklearn for Breast Cancer Dataset 0.9181286549707602


In [2]:
class Decision_tree:
    def __init__(self):
        self.ypred = None
    
    def fit(self,X,Y):
        self.tree = Tree()(X,Y)
        return self
    
    def predict(self,X):
        y_pred = np.empty(len(X))
        for row in range(len(X)):
            query = X[row]
            pred  = self.tranverse_tree(query)
            y_pred[row] = pred
        self.ypred = y_pred
        print(f"prediction is {y_pred}")
        return self
    
    def score(self,Y):
        return np.mean(self.ypred == Y)
    
    def tranverse_tree(self,query):
        tree_dict = self.tree.__dict__
        while tree_dict and not tree_dict['is_leaf']:
            deciding_feature = tree_dict['feature']
            split_rule       = tree_dict['split_rule']
            if query[deciding_feature] < split_rule:
                tree_dict = tree_dict['left_child'].__dict__
            else:
                tree_dict = tree_dict['right_child'].__dict__
        return tree_dict['leaf']

    
class Tree:
    def __init__(self):
        self.is_leaf     = False
        self.left_child  = None
        self.right_child = None
        self.feature     = None
        self.split_rule  = None
        
    def __call__(self,X,Y):
        if not self.is_leaf:
            return self.split(X,Y)
    
    def split(self,X,Y):
        n_samples,n_features = X.shape
        positive_class       = Y[0]
        lowest_gini          = math.inf
        best_split           = []
        
        if self.stop_criterion(X,Y):
            self.leaf = Y[0]
            self.is_leaf = True
            return self
        else:
            for i in range(n_features):
                feat          = X[:,i]
                unique_values = np.unique(feat)
                mids          = self.midpoints(unique_values)
                 
                for val in mids:
                    split           = self._make_rule(X,i,val)
                    below_criterion = Y[split]
                    above_criterion = Y[~split]

                    below_posi_class_prob = np.sum(below_criterion == positive_class)/len(below_criterion)
                    below_neg_class_prob  = 1 -  below_posi_class_prob

                    above_posi_class_prob = np.sum(above_criterion == positive_class)/len(above_criterion)
                    above_posi_class_prob  = 1 -  above_posi_class_prob

                    below_gini = 1 -  below_posi_class_prob**2 - below_neg_class_prob**2
                    above_gini = 1 -  above_posi_class_prob**2 - above_posi_class_prob**2

                    weigh_below  = len(below_criterion)/len(Y)
                    weigh_above  = len(above_criterion)/len(Y)

                    weighted_gini = weigh_below*below_gini + weigh_above * above_gini

                    if weighted_gini < lowest_gini:
                        lowest_gini = weighted_gini
                        best_split  = split
                        self.feature     = i
                        self.split_rule  = val
        
        if len(X[best_split]) == len(Y[best_split]):           
            self.left_child  = Tree()(X[best_split], Y[best_split])
            self.right_child = Tree()(X[~best_split], Y[~best_split])
        return self
    
    @staticmethod
    def midpoints(x):
        #returns a array with midvalues of 2 consequtive array elements
        mid = np.empty(len(x)-1)
        for i in range(len(x)-1):
            mid[i] = (x[i] + x[i+1])/2
        return mid
        
        
    @staticmethod
    def stop_criterion(X,Y):
        #there is only 1 sample left
        if len(X) == 1 or len(Y) == 1:
            return True
        # data belongs to the same class
        if np.prod(Y == Y[0]):
            return True
        return False
        
    @staticmethod
    def _make_rule (X,idx , val):
        # return the splitting rule ( univariate splits for numerical data )
        return X[:, idx] < val

In [3]:
cf = Decision_tree()
cf = cf.fit(X_train_i,y_train_i)
cf.predict(X_test_i)
print(f"Accuracy score by Self Implementation for Iris Dataset {cf.score(y_test_i)}\n")

cf = Decision_tree()
cf = cf.fit(X_train_b,y_train_b)
cf.predict(X_test_b)
print(f"Accuracy score by Self Implementation for Breast Cancer Dataset {cf.score(y_test_b)}")

prediction is [0. 0. 2. 1. 2. 0. 2. 0. 2. 2. 2. 2. 2. 1. 0. 1. 2. 2. 0. 2. 0. 2. 0. 2.
 0. 0. 1. 2. 1. 0. 2. 0. 0. 2. 2. 0. 0. 2. 0. 2. 0. 1. 0. 2. 2.]
Accuracy score by Self Implementation for Iris Dataset 0.8666666666666667

prediction is [1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1.
 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1.
 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0.
 0. 1. 1.]
Accuracy score by Self Implementation for Breast Cancer Dataset 0.8421052631578947


### Sample Node of Tree

In [4]:
cf.tree.__dict__

{'is_leaf': False,
 'left_child': <__main__.Tree at 0x1890b44fa48>,
 'right_child': <__main__.Tree at 0x1890b448388>,
 'feature': 9,
 'split_rule': 0.055349999999999996}

## Comparision
The Scores produced by sklearn was better
- Iris Dataset, Sklearn =  0.9555555555555556, Self = 0.8666666666666667
- Breast Cancer Dataset = 0.9181286549707602, Self = 0.8421052631578947